In [1]:
import jieba

sample_str = "所以，我在每一个无法入睡的夜晚，像小时候一样，攥紧拳头，" \
             "捂紧被子不让哭泣的声音泄露在黑暗里，等哭累了，也就慢慢睡着了，" \
             "我怕招人厌恶，我怕恶语相加，我怕看到他不耐烦和厌恶的表情。"
# 分词演示
pre_process = sample_str.split('，')

print(pre_process)
for item in pre_process:
    seg_list = jieba.cut(item)
    print(", ".join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/14/tgjmqxwd3mq_dmn00xtb25lw0000gn/T/jieba.cache


['所以', '我在每一个无法入睡的夜晚', '像小时候一样', '攥紧拳头', '捂紧被子不让哭泣的声音泄露在黑暗里', '等哭累了', '也就慢慢睡着了', '我怕招人厌恶', '我怕恶语相加', '我怕看到他不耐烦和厌恶的表情。']


Loading model cost 0.327 seconds.
Prefix dict has been built successfully.


所以
我, 在, 每, 一个, 无法, 入睡, 的, 夜晚
像, 小时候, 一样
攥, 紧, 拳头
捂紧, 被子, 不让, 哭泣, 的, 声音, 泄露, 在, 黑暗, 里
等, 哭, 累, 了
也, 就, 慢慢, 睡着, 了
我怕, 招人, 厌恶
我怕, 恶语相加
我怕, 看到, 他, 不耐烦, 和, 厌恶, 的, 表情, 。


In [2]:
# 命名实体识别演示
import spacy

sample2_str = "最近在看佛洛依德、卡尔荣格的书，但是我不承认我心里的残缺面，在2018年患病的初期，我真的特别感激我的男朋友，" \
              "陪我去北京看病，容忍了一次又一次我情绪最差时的恶劣态度，我甚至觉得，" \
              "他是这个世界上唯一能用心感受我所经历的痛苦的人。但是，的确，" \
              "这个世界上是不存在感同身受的，我知道他努力过了，也曾经给过我很大的希望。现在，我想他也疲倦了。" \
              "写于2020年3月28日，于重庆"

nlp = spacy.load("zh_core_web_sm")
doc = nlp(sample2_str)
for entry in doc.ents:
    print(entry.text, entry.label_)

佛洛依德 ORG
卡尔荣格 GPE
2018年 DATE
北京 GPE
2020年3月28日 DATE
重庆 GPE


In [3]:
# 文本分类
import fasttext

classifier = fasttext.train_supervised(input='shuffle_data.txt', dim=100, epoch=1000, lr=0.1, wordNgrams=2, loss='softmax')
classifier.save_model('classifier.model')

Read 0M words
Number of words:  11
Number of labels: 2
Progress: 100.0% words/sec/thread:   42199 lr:  0.000000 avg.loss:  0.020128 ETA:   0h 0m 0s


In [9]:
print(classifier.predict("有时候太难受的时候, 有点想一了百了", k=2))
print(classifier.predict("一了百了并不是解决问题的办法", k=2))

(('__label__1,', '__label__2,'), array([0.68634456, 0.31367546]))
(('__label__1,', '__label__2,'), array([0.76292688, 0.23709312]))


In [4]:
# 文本相似度计算
import spacy

def check_zh_doc_similarity():
    nlp = spacy.load('zh_core_web_sm')
    doc1 = nlp('你好吗?')
    doc2 = nlp('你还好吗?')
    doc3 = nlp('今天你还好吗?')
    doc4 = nlp('你的身体今天还好吗?')
    print(doc1.similarity(doc2))
    print(doc2.similarity(doc3))
    print(doc1.similarity(doc3))
    print(doc1.similarity(doc4))

In [5]:
check_zh_doc_similarity()

0.7276577012401315
0.9146431706372442
0.628690517163414
0.5599675731711398


/var/folders/14/tgjmqxwd3mq_dmn00xtb25lw0000gn/T/ipykernel_27993/905372968.py:10: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(doc1.similarity(doc2))
/var/folders/14/tgjmqxwd3mq_dmn00xtb25lw0000gn/T/ipykernel_27993/905372968.py:11: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-se

In [14]:
# 主题模型
from gensim import corpora, models
import jieba.posseg as jp, jieba

# 文本集
texts = [
    '美国教练坦言，没输给中国女排，是输给了郎平',
    '美国无缘四强，听听主教练的评价',
    '中国女排晋级世锦赛四强，全面解析主教练郎平的执教艺术',
    '为什么越来越多的人买MPV，而放弃SUV？跑一趟长途就知道了',
    '跑了长途才知道，SUV和轿车之间的差距',
    '家用的轿车买什么好']
jieba.add_word('四强', 9, 'n')
flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')  # 词性
stopwords = ('没', '就', '知道', '是', '才', '听听', '坦言', '全面', '越来越', '评价', '放弃', '人')
words_ls = []
for text in texts:
    words = [word.word for word in jp.cut(text) if word.flag in flags and word.word not in stopwords]
    words_ls.append(words)
print(words_ls)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/14/tgjmqxwd3mq_dmn00xtb25lw0000gn/T/jieba.cache
Loading model cost 0.298 seconds.
Prefix dict has been built successfully.


[['美国', '输给', '中国女排', '输给', '郎平'], ['美国', '无缘', '四强', '主教练'], ['中国女排', '晋级', '世锦赛', '四强', '主教练', '郎平', '执教', '艺术'], ['买', 'MPV', 'SUV', '跑', '长途'], ['跑', '长途', 'SUV', '轿车', '差距'], ['家用', '轿车', '买']]


In [15]:
# 去重，存到字典
dictionary = corpora.Dictionary(words_ls)
print(dictionary)

Dictionary<19 unique tokens: ['中国女排', '美国', '输给', '郎平', '主教练']...>


In [21]:
corpus = [dictionary.doc2bow(words) for words in words_ls]
# print(corpus)
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
for topic in lda.print_topics(num_words=4):
    print(topic)

(0, '0.079*"轿车" + 0.074*"输给" + 0.071*"郎平" + 0.070*"中国女排"')
(1, '0.081*"四强" + 0.076*"主教练" + 0.072*"美国" + 0.066*"跑"')


In [22]:
# 前面设置了num_topics = 2 所以这里有两个主题，显然第一个是汽车相关topic，第二个是体育相关topic

In [23]:
# 主题推断
print(lda.inference(corpus))

(array([[5.313778  , 0.6862094 ],
       [0.6112416 , 4.3887486 ],
       [7.9553366 , 1.0446328 ],
       [1.14796   , 4.8520236 ],
       [5.226868  , 0.77311647],
       [3.376752  , 0.6232384 ]], dtype=float32), None)


In [24]:
text5 = '中国女排将在郎平的率领下向世界女排三大赛的三连冠发起冲击'
bow = dictionary.doc2bow([word.word for word in jp.cut(text5) if word.flag in flags and word.word not in stopwords])
ndarray = lda.inference([bow])[0]
print(text5)
for e, value in enumerate(ndarray[0]):
    print('\t主题%d推断值%.2f' % (e, value))

中国女排将在郎平的率领下向世界女排三大赛的三连冠发起冲击
	主题0推断值2.38
	主题1推断值0.62


In [37]:
# 文本摘要
# https://ai.baidu.com/tech/nlp_apply/news_summary

In [ ]:
# 情感识别
# https://ai.baidu.com/tech/nlp_apply/sentiment_classify